In [1]:
from scipy.optimize import root
import numpy as np

In [2]:
params = {
    'eps': 1.5,
    'tht': 1,
    'alf': .5,
    'bet': .9,
    'om_0': 1,
    'om_1': 1,
    'om_2': 1,
    'delt': .05
}

In [3]:
ss_shock = {
    'a_0': 0,
    'a_1': 0,
    'a_2': 0
}

In [4]:
shock_params = {
    'var_a_0': .5,
    'var_a_1': .6,
    'var_a_2': .7,
    'rho_a_0': .9,
    'rho_a_1': .9,
    'rho_a_2': .9
}

In [5]:
ss_params = {**params, **ss_shock}
model_params = {**params, **shock_params}

In [6]:
model_params

{'eps': 1.5,
 'tht': 1,
 'alf': 0.5,
 'bet': 0.9,
 'om_0': 1,
 'om_1': 1,
 'om_2': 1,
 'delt': 0.05,
 'var_a_0': 0.5,
 'var_a_1': 0.6,
 'var_a_2': 0.7,
 'rho_a_0': 0.9,
 'rho_a_1': 0.9,
 'rho_a_2': 0.9}

In [7]:
print(','.join(f'{key}' for key in ss_params))

eps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2


In [8]:
print(','.join(f'{key}' for key in model_params))

eps,tht,alf,bet,om_0,om_1,om_2,delt,var_a_0,var_a_1,var_a_2,rho_a_0,rho_a_1,rho_a_2


In [9]:
def assign_params(ss_params):
    a_0 = ss_params['a_0']
    a_1 = ss_params['a_1']
    a_2 = ss_params['a_2']
    alf = ss_params['alf']
    delt = ss_params['delt']
    om_0 = ss_params['om_0']
    om_1 = ss_params['om_1']
    om_2 = ss_params['om_2']
    eps = ss_params['eps']
    tht = ss_params['tht']
    bet = ss_params['bet']
   
    return eps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2



In [10]:
def equations(vars, params):
	n_0,n_1,n_2,k,c_0,c_1,c_2 = vars
	eps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2 = assign_params(params)
	ph=(om_0*c_0**((eps-1)/eps)+om_1*c_1**((eps-1)/eps)+om_2*c_2**((eps-1)/eps)	)**((1-tht*eps)/(eps-1))

	v_0=om_0*c_0**(-1/eps)*ph
	v_1=om_1*c_1**(-1/eps)*ph
	v_2=om_2*c_2**(-1/eps)*ph

	fk_0=np.exp(a_0)*n_0*alf*k**(alf-1)
	fk_1=np.exp(a_1)*n_1*alf*k**(alf-1)
	fk_2=np.exp(a_2)*n_2*alf*k**(alf-1)

	fn_0=np.exp(a_0)*(1-alf)*k**(alf)
	fn_1=np.exp(a_1)*(1-alf)*k**(alf)
	fn_2=np.exp(a_2)*(1-alf)*k**(alf)

	lamk=v_0*fk_0/n_0
	eq1=v_1*fk_1-n_1*lamk
	eq2=v_2*fk_2-n_2*lamk
 
	#changed
	eq1=v_1*fn_1-v_0*fn_0
	eq2=v_2*fn_2-v_0*fn_0
 

	eq3=bet*(fk_0+1-delt)-1
	eq4=n_0+n_1+n_2-1
	eq5=c_0-np.exp(a_0)*n_0*k**alf-(1-delt)*k+k
	eq6=c_1-np.exp(a_1)*n_1*k**alf
	eq7=c_2-np.exp(a_2)*n_2*k**alf
 
 
	return [eq1, eq2, eq3, eq4, eq5, eq6, eq7]


In [11]:
initial_guess = [.3,.3,.3,.3,.3,.3,.3]
def solve_equations(params):
    
    sol = root(equations, initial_guess, args=(ss_params,))
    
    # Extract the solution vector
    n_0,n_1,n_2,k,c_0,c_1,c_2 = sol.x
    a_0=0
    a_1=0
    a_2=0
    ss_dict = {
    'n_0': n_0,
    'n_1': n_1,
    'n_2': n_2,
    'k': k,
    'c_0': c_0,
    'c_1': c_1,
    'c_2': c_2,
    'a_0': a_0,
    'a_1': a_1,
    'a_2': a_2
    }
    return sol, ss_dict
    

# Call the wrapper function
sol, ss_dict=solve_equations(ss_params)
print(sol)
ss_dict


 message: The solution converged.
 success: True
  status: 1
     fun: [ 2.729e-11  2.729e-11 -2.434e-11  0.000e+00 -2.954e-11
            2.780e-11 -7.916e-12]
       x: [ 3.718e-01  3.141e-01  3.141e-01  1.331e+00  3.624e-01
            3.624e-01  3.624e-01]
    nfev: 23
    fjac: [[-2.063e-03 -2.063e-03 ... -4.522e-02 -4.525e-02]
           [ 2.632e-03  2.632e-03 ...  6.417e-01  4.002e-02]
           ...
           [ 5.242e-01 -7.287e-01 ... -1.888e-01 -1.888e-01]
           [ 5.233e-01 -1.325e-01 ...  3.607e-01  3.607e-01]]
       r: [-1.430e+00 -6.939e-01 ...  3.076e-01  4.434e-01]
     qtf: [-1.386e-12  8.484e-11 -6.690e-11 -2.488e-10 -1.740e-10
           -2.156e-10  4.120e-10]


{'n_0': 0.37179487184553434,
 'n_1': 0.31410256406175696,
 'n_2': 0.3141025640927087,
 'k': 1.3313609475553958,
 'c_0': 0.36242603562185355,
 'c_1': 0.36242603559389786,
 'c_2': 0.3624260355938979,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [12]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('var \n')
        for key, value in ss_dict.items():
            f.write(f'{key}\n')
        f.write(f';\n\n\n')
        f.write('varexo \n')
        for key, value in ss_shock.items():
            f.write(f'{key}_shock\n')
        f.write(f';\n\n\n')
# Call the function
write_dict_to_file(ss_dict, 't_vars.m')

In [13]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('initval;\n')
        for key, value in ss_dict.items():
            f.write(f'{key}={value};\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_ss.m')

In [14]:
def write_dict_to_file(params, filename):
    
    with open(filename, 'w') as f:
        f.write('parameters ')
        f.write(','.join(f'{key}' for key in params))
        f.write(';\n\n')
        for key, value in params.items():
            f.write(f'{key}={value};\n')

# Call the function
write_dict_to_file(model_params, 't_parameters.m')

In [15]:
def write_dict_to_file(ss_dict, filename):
    #variance_list = list(shock_params.values())[:3]
    with open(filename, 'w') as f:
        f.write('shocks;\n')
        i=0
        for key, value in ss_shock.items():
            f.write(f'var {key}_shock=var_{key};\n')
            i+=1
        f.write(f'\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_shocks.m')